In [1]:
# Import required libraries
import numpy as np
import math
import datetime as dt
import pandas as pd
import pathlib

In [2]:
# Load data
# get relative data folder
PATH = pathlib.Path('./data').parent
DATA_PATH = PATH.joinpath("data").resolve()
PIDAR = pd.read_csv(DATA_PATH.joinpath("pidar.csv"))
AT = pd.read_csv(DATA_PATH.joinpath("DS_Asistencia_Tecnica.csv"))
EVA = pd.read_csv(DATA_PATH.joinpath('EVA_cultivos_modificado.csv'))
url = 'https://raw.githubusercontent.com/ds4a-team-envigado/project/master/Dashboard/VERSION%202/data/AP_SABANAGENERAL.csv'
SG = pd.read_csv(url, index_col=0,sep=";",encoding='latin-1')
dfs = [PIDAR,AT,EVA,SG]

In [3]:
# #Agrupar por departamtento cada una de las columnas
dff = EVA.groupby(['DEPARTAMENTO','YEAR','CADENA_PRODUCTIVA_ADR']).sum().reset_index()
dff.head()

,DEPARTAMENTO,YEAR,CADENA_PRODUCTIVA_ADR,COD_DEP,COD_MUN,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO,NIVEL,ID_ADR
0,AMAZONAS,2006,ARROZ,273,274339,11,11,16,4.35,9,37527
1,AMAZONAS,2006,MAIZ,455,457151,204,203,468,7.86,15,62520
2,AMAZONAS,2007,ACHICORIA,182,182541,6,6,6,2.00,6,21610
3,AMAZONAS,2007,AHUYAMA,91,91540,10,10,38,3.80,3,11213
4,AMAZONAS,2007,AJI,182,182541,28,28,35,2.50,6,22418


In [4]:
PIDAR.head()

,Unnamed: 0,Unnamed: 0.1,BP,No. VP,Nombre Proyecto,Departamento,Municipio,Total beneficiarios,Hombres,Mujeres,...,Valor total cofinanciación ADR,Valor Contrapartida,Valor total proyecto,Hectáreas,Resolución,Fecha resolución,Vigencia,Lineamiento de consejo directivo,AP_ID_ADR,AP_CADENA_PRODUCTIVA_ADR
0,5,5,6,6,Mejorar las capacidades técnicas de adecuación...,CHOCO,CARMEN DEL DARIÉN,35,30,5,...,250000000,513500000,763500000,100,1219,06/29/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
1,16,16,17,17,Fortalecer la capacidad productiva y manejo de...,META,MESETAS,40,31,9,...,500000000,106600000,606600000,40,1250,07/12/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
2,19,19,20,20,Fortalecer la productividad y competitividad d...,NORTE DE SANTANDER,TIBÚ,46,31,15,...,252785478,351466000,604251478,92,1453,10/12/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
3,22,22,23,23,Fortalecer la cadena productiva de café y plát...,ANTIOQUIA,DABEIBA,25,20,5,...,401373250,355140000,756513250,69,1351,08/28/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
4,24,24,25,25,Fortalecimiento de la productividad del cultiv...,META,LA MACARENA,28,16,12,...,255000000,41570000,296570000,93,1401,09/19/2017 12:00:00 AM,2017,NaN,1,AGRICOLA


In [5]:
EVA.head()

,COD_DEP,DEPARTAMENTO,COD_MUN,MUNICIPIO,GRUP_CULTIVO,SUBGRUP_CULTIVO,CULTIVO,DESAGREGACION_SIST_PRODUCTIVO,YEAR,PERIODO,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO,ESTADO_PRODUCCION,NOMBRE_CIENTI,CICLO_CULTIVO,CADENA_PRODUCTIVA_ADR,NIVEL,ID_ADR
0,15,BOYACA,15114,BUSBANZA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,1,1,1.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
1,25,CUNDINAMARCA,25754,SOACHA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,82,80,1440,18.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
2,25,CUNDINAMARCA,25214,COTA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,2,26,17.33,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
3,54,NORTE DE SANTANDER,54405,LOS PATIOS,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,3,3,48,16.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
4,54,NORTE DE SANTANDER,54518,PAMPLONA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,1,1,5,10.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807


## Preprocesamiento

In [6]:
import gzip
import json
import string

import nltk # imports the natural language toolkit
import pandas as pd
import plotly as px
import matplotlib as plt

import nltk # imports the natural language toolkit
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import numpy  as np
import string
import plotly
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer

import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

vec = CountVectorizer()

nltk.download('punkt')

CP_ADR = EVA['CADENA_PRODUCTIVA_ADR'].unique()

[nltk_data] Downloading package punkt to C:\Users\Carlos Loaiza
[nltk_data]     Velez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Carlos Loaiza
[nltk_data]     Velez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Carlos Loaiza
[nltk_data]     Velez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [8]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    
    s = s.upper()
    return s    

In [9]:
# import re
# from unicodedata import normalize

# word_list = []

# for sentence in PIDAR['Nombre Proyecto']:
#     word_list = word_list + re.split(r',', sentence)
    
# word_list2 = []
# for word in word_list:
#     word = word.lstrip().rstrip().upper()
#     word = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", word), 0, re.I)
#     word_list2.append(word)

# word_list2 = list(dict.fromkeys(word_list2))
# word_list2.sort()

In [19]:
# clean text data
PIDAR['TITULO'] = PIDAR['Nombre Proyecto'].apply(lambda x: clean_text(x))

In [20]:
new_df_PIDAR = []
for i in PIDAR['Nombre Proyecto'].unique():
    list_w = i.split(" ")
    temp = PIDAR[PIDAR['Nombre Proyecto']==i].copy()
    for j in list_w:
        w = normalize(j)
        if w in CP_ADR:
            temp['CADENA_PRODUCTIVA_ADR']=w
            new_df_PIDAR.append(temp)
            
new_df_PIDAR=pd.concat(new_df_PIDAR)
new_df_PIDAR.shape

(83, 25)

In [21]:
len(new_df_PIDAR['BP'].unique())

63

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html?highlight=merge

In [22]:
new_df_PIDAR.head()

,Unnamed: 0,Unnamed: 0.1,BP,No. VP,Nombre Proyecto,Departamento,Municipio,Total beneficiarios,Hombres,Mujeres,...,Valor total proyecto,Hectáreas,Resolución,Fecha resolución,Vigencia,Lineamiento de consejo directivo,AP_ID_ADR,AP_CADENA_PRODUCTIVA_ADR,TITULO,CADENA_PRODUCTIVA_ADR
0,5,5,6,6,Mejorar las capacidades técnicas de adecuación...,CHOCO,CARMEN DEL DARIÉN,35,30,5,...,763500000,100,1219,06/29/2017 12:00:00 AM,2017,NaN,1,AGRICOLA,mejorar las capacidades técnicas de adecuación...,PLATANO
1,16,16,17,17,Fortalecer la capacidad productiva y manejo de...,META,MESETAS,40,31,9,...,606600000,40,1250,07/12/2017 12:00:00 AM,2017,NaN,1,AGRICOLA,fortalecer la capacidad productiva manejo de p...,CAFE
3,22,22,23,23,Fortalecer la cadena productiva de café y plát...,ANTIOQUIA,DABEIBA,25,20,5,...,756513250,69,1351,08/28/2017 12:00:00 AM,2017,NaN,1,AGRICOLA,fortalecer la cadena productiva de café plátan...,CAFE
4,24,24,25,25,Fortalecimiento de la productividad del cultiv...,META,LA MACARENA,28,16,12,...,296570000,93,1401,09/19/2017 12:00:00 AM,2017,NaN,1,AGRICOLA,fortalecimiento de la productividad del cultiv...,CAUCHO
5,30,30,31,31,Mejorar las capacidades productivas del cultiv...,CAUCA,CALDONO,51,40,11,...,600939000,51,1438,10/06/2017 12:00:00 AM,2017,NaN,1,AGRICOLA,mejorar las capacidades productivas del cultiv...,CAFE


In [26]:
EVA.head()

,COD_DEP,DEPARTAMENTO,COD_MUN,MUNICIPIO,GRUP_CULTIVO,SUBGRUP_CULTIVO,CULTIVO,DESAGREGACION_SIST_PRODUCTIVO,YEAR,PERIODO,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO,ESTADO_PRODUCCION,NOMBRE_CIENTI,CICLO_CULTIVO,CADENA_PRODUCTIVA_ADR,NIVEL,ID_ADR
0,15,BOYACA,15114,BUSBANZA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,1,1,1.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
1,25,CUNDINAMARCA,25754,SOACHA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,82,80,1440,18.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
2,25,CUNDINAMARCA,25214,COTA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,2,2,26,17.33,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
3,54,NORTE DE SANTANDER,54405,LOS PATIOS,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,3,3,48,16.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807
4,54,NORTE DE SANTANDER,54518,PAMPLONA,HORTALIZAS,ACELGA,ACELGA,ACELGA,2006,2006B,1,1,5,10.00,FRUTO FRESCO,BETA VULGARIS,TRANSITORIO,ACELGA,3,10807


In [ ]:
'AREA_SEMBR','AREA_COSECH','PRODUCCION','RENDIMIENTO','CICLO_CULTIVO'

In [39]:
EVA2 = EVA.groupby(['DEPARTAMENTO','CADENA_PRODUCTIVA_ADR'])[['AREA_SEMBR','AREA_COSECH','PRODUCCION','RENDIMIENTO']].mean().reset_index()
EVA2.columns = ['Departamento','CADENA_PRODUCTIVA_ADR','AREA_SEMBR','AREA_COSECH','PRODUCCION','RENDIMIENTO']
EVA2.head()

,Departamento,CADENA_PRODUCTIVA_ADR,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO
0,AMAZONAS,ACHICORIA,4.000000,4.000000,4.000000,1.000000
1,AMAZONAS,AGUACATE,21.875000,14.125000,84.125000,5.752500
2,AMAZONAS,AGUAJE,25.611111,18.666667,234.833333,11.279444
3,AMAZONAS,AHUYAMA,11.333333,11.333333,43.000000,3.793333
4,AMAZONAS,AJI,9.294118,8.558824,6.500000,0.807059


In [47]:
PRUEBA = pd.merge(new_df_PIDAR, EVA2, on = ['CADENA_PRODUCTIVA_ADR','Departamento'],how='left')
PRUEBA.head()

,Unnamed: 0,Unnamed: 0.1,BP,No. VP,Nombre Proyecto,Departamento,Municipio,Total beneficiarios,Hombres,Mujeres,...,Vigencia,Lineamiento de consejo directivo,AP_ID_ADR,AP_CADENA_PRODUCTIVA_ADR,TITULO,CADENA_PRODUCTIVA_ADR,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO
0,5,5,6,6,Mejorar las capacidades técnicas de adecuación...,CHOCO,CARMEN DEL DARIÉN,35,30,5,...,2017,NaN,1,AGRICOLA,mejorar las capacidades técnicas de adecuación...,PLATANO,739.367403,616.287293,5346.646409,6.472403
1,16,16,17,17,Fortalecer la capacidad productiva y manejo de...,META,MESETAS,40,31,9,...,2017,NaN,1,AGRICOLA,fortalecer la capacidad productiva manejo de p...,CAFE,239.132075,194.056604,187.836478,0.845419
2,22,22,23,23,Fortalecer la cadena productiva de café y plát...,ANTIOQUIA,DABEIBA,25,20,5,...,2017,NaN,1,AGRICOLA,fortalecer la cadena productiva de café plátan...,CAFE,1424.058235,1184.952684,1276.032757,0.996113
3,24,24,25,25,Fortalecimiento de la productividad del cultiv...,META,LA MACARENA,28,16,12,...,2017,NaN,1,AGRICOLA,fortalecimiento de la productividad del cultiv...,CAUCHO,780.493151,80.100457,126.735160,1.576149
4,30,30,31,31,Mejorar las capacidades productivas del cultiv...,CAUCA,CALDONO,51,40,11,...,2017,NaN,1,AGRICOLA,mejorar las capacidades productivas del cultiv...,CAFE,2590.627297,2108.002625,2034.099738,0.898184


In [48]:
len(PRUEBA)

83

In [49]:
len(new_df_PIDAR)

83

In [52]:
PRUEBA.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'BP', 'No. VP', 'Nombre Proyecto',
       'Departamento', 'Municipio', 'Total beneficiarios', 'Hombres',
       'Mujeres', 'Total Víctimas', 'Valor cofinanciado ADR',
       'Valor encargo fiduciario', 'Valor total cofinanciación ADR',
       'Valor Contrapartida', 'Valor total proyecto', 'Hectáreas',
       'Resolución', 'Fecha resolución', 'Vigencia',
       'Lineamiento de consejo directivo', 'AP_ID_ADR',
       'AP_CADENA_PRODUCTIVA_ADR', 'TITULO', 'CADENA_PRODUCTIVA_ADR',
       'AREA_SEMBR', 'AREA_COSECH', 'PRODUCCION', 'RENDIMIENTO'],
      dtype='object')

In [84]:
import plotly.express as px
fig = px.scatter(PRUEBA, x='RENDIMIENTO', y='Valor total cofinanciación ADR', color='CADENA_PRODUCTIVA_ADR',
                 size='Total beneficiarios', hover_data=['BP','Departamento','Vigencia'], title = 'Valor confinanciado vs rendimiento por proyecto')
fig.show()

In [105]:
EVA3 = EVA.groupby(['YEAR','CADENA_PRODUCTIVA_ADR'])[['RENDIMIENTO']].mean().reset_index()
fig = px.line(EVA3, x='YEAR', y='RENDIMIENTO', color='CADENA_PRODUCTIVA_ADR', title='Life expectancy in Canada')
fig.show()

In [106]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

le = preprocessing.LabelEncoder()
scaler = StandardScaler()


points = PRUEBA[['RENDIMIENTO', 'Valor total cofinanciación ADR','Total beneficiarios']]
points = points.fillna(0)
scaler.fit(points)
points = scaler.transform(points)
points= pd.DataFrame(points, columns = ['RENDIMIENTO', 'Valor total cofinanciación ADR','Total beneficiarios'])
# create kmeans object
model = KMeans(n_clusters=4)
# fit kmeans object to data
model.fit(points)
# print location of clusters learned by kmeans object
print(model.cluster_centers_)
# save new clusters for chart
y_km = model.fit_predict(points)
points['y_km']=y_km
PRUEBA['y_km']=y_km
fig = px.scatter(points, x='RENDIMIENTO', y='Valor total cofinanciación ADR', color='y_km'
                     ,template="simple_white", title = "Clouster")
fig.show()

[[-0.11509986  1.2679382   0.62795341]
 [-0.34604649 -0.51050411 -0.48596355]
 [ 3.20795859 -0.77574547 -0.9766291 ]
 [ 0.42581157  1.36948146  2.23338294]]
